# **Music Theme Recognition**

Chuang Caleb
20204134

## **Package Imports**

In [1]:
import pandas as pd
import numpy as np
# from skmultilearn.dataset import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, hamming_loss, f1_score
from skmultilearn.adapt import MLkNN
from imblearn.under_sampling import ClusterCentroids

## **Choosing Parameters**

In [2]:
target_label = 'wonder'
feature_start = 'Number_of_Pitches'
feature_end = 'Metrical_Diversity'
k_value = 3
rand_state = 10

## **Loading Data**

### Importing Dataset

#### Features

In [3]:
# Access song_theme_feature_database
song_theme_feature_database_path = '../data/features/song_theme_feature_database.csv'
raw_feature_df = pd.read_csv(song_theme_feature_database_path)


#### Labels

In [4]:
# Access song_theme_labels_database
song_theme_label_database_path = '../data/song_theme_label_database.xlsx'
raw_label_df = pd.read_excel(song_theme_label_database_path)

### Prepare datasets

Select the features for this iteration

In [5]:
# Feature Selection
feature_df = raw_feature_df.loc[:,feature_start:feature_end]


Select the labels for this iteration

In [6]:
# Filter for only recognized samples in labels_df
main_label_df = raw_label_df[raw_label_df.recognizable == 1]

# Filter out unused labels and metadata
main_label_df.drop(main_label_df.columns[19:], axis=1, inplace=True) # unused labels
main_label_df.drop(main_label_df.columns[0:4], axis=1, inplace=True) # metadata

label_df = main_label_df[[target_label]]


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [7]:
dum_label_df = raw_label_df[raw_label_df.recognizable == 1]
comp_df = pd.DataFrame({'label': dum_label_df['sample'].reset_index(
    drop=True), 'feat': raw_feature_df['sample']})

mismatch_df = comp_df[comp_df.label != comp_df.feat]

Convert to numpy type

In [8]:
feature_np = feature_df.to_numpy()
label_np= label_df.to_numpy().astype(int)


Get list of samples

In [9]:
sample_list = raw_feature_df['sample']


### Prepare variables

In [10]:
(x_train, x_test, y_train, y_test, ids_train, ids_test) = train_test_split(
    feature_np, label_np, sample_list, test_size=0.2, random_state=rand_state)

In [11]:
# from collections import Counter

# cc = ClusterCentroids(random_state=0)
# X_resampled, y_resampled = cc.fit_resample(x_train, y_train)
# print(sorted(Counter(y_resampled).items()))

# my_list = raw_label_df[raw_label_df.recognizable ==
#                            1].loc[:, ['sample', 'risk']]


## Preparing Model

In [12]:
classifier = MLkNN(k=k_value)

# train
classifier.fit(x_train, y_train)
# classifier.fit(X_resampled, y_resampled)

MLkNN(k=3)

In [13]:
# predict
# predictions = classifier.predict(x_test)
predictions = classifier.predict(x_test).asformat('array')

# scores = cross_val_score(classifier, x_test, y_test, cv=3, scoring='accuracy')

print('Accuracy: ' + str(accuracy_score(y_test, predictions)))
print('Haming Loss: ' +  str(hamming_loss(y_test, predictions)))
print('F1-score: ' +  str(f1_score(y_test, predictions)))


flat_predictions = [val for sublist in predictions for val in sublist]
flat_y_true = [val for sublist in y_test for val in sublist]
# print(flat_predictions)
# print(flat_y_true)



Accuracy: 0.6923076923076923
Haming Loss: 0.3076923076923077
F1-score: 0.5


## Evaluation

In [14]:
results = pd.DataFrame(
    {'sample': ids_test, 'y_true': flat_y_true, 'y_pred': flat_predictions})

results = results.sort_index()

# some_list = my_list.merge({'y_true':flat_y_true})

# my_list[my_list['y_true'].notna()]
# some_list

# my_list = raw_label_df[raw_label_df.recognizable ==
#                        1].loc[:, ['sample', 'love']]

# final = results.merge(my_list, on='sample')

# my_list
# final
results

,sample,y_true,y_pred
3,abba-dancing_queen_bitmidi.mid,0,0
5,ABBA.Angel Eyes K_bitmidi.mid,0,0
6,ABBA.Chiquitita K_bitmidi.mid,0,1
8,ABBA.Happy new year_bitmidi.mid,0,0
15,Ain't-No-Mountain-High-Enough_bitmidi.mid,0,0
17,Aladdin(AFriendLikeMe)_bitmidi.mid,1,0
23,AMRINGER.Can't help falling in love_bitmidi.mid,1,1
25,Another-One-Bites-The-Dust-1_bitmidi.mid,0,0
31,avicii-hey_brother_bitmidi.mid,0,1
34,B.TYLER.HoldinG out for hero K_bitmidi.mid,1,0
